In [ ]:
import quantum_engine as qe
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# NOTE:
# "INPUT" is for class and objects
# "config" is for other settings that is not class and objects

my_teacher_model = qe.teacher_model(verbose=False)
my_teacher_model.config(select_model="quad")
my_teacher_model.generate_training_batch()
my_teacher_model.plot_training_batch()

In [ ]:
my_loss_function = qe.loss_function(verbose=False)
my_loss_function.config(select_loss = "quad_loss")

In [ ]:
my_quantum_encoding = qe.quantum_encoding(verbose=False)
my_quantum_encoding.config(select_encoding="default")

In [ ]:
# Haven make it quantum engine
my_entangling = qe.quantum_block(verbose=False)
my_entangling.config(select_block="ham")
config_ham_dict = {"ham_type": "ham", }
my_entangling.config_dict(config_ham_dict)

In [ ]:
my_parameterised = qe.quantum_block(verbose=False)
my_parameterised.config(select_block="qubit_rotate")
config_qubit_rotate_dict = {"qubit_rotate_type": "xyz"}
my_parameterised.config_dict(config_qubit_rotate_dict)

In [ ]:
# TO BE DELETED
my_entangling = qe.quantum_ham_layer(verbose=False)
my_entangling.config(select_ham_type = "Ising")
my_parameterised = qe.quantum_ml_layer(verbose=False)
my_parameterised.config(select_ml_type= " Single Qubit Rotations/ Customs")
my_reupload = qe.quantum_ul_layer(verbose=False)
my_reupload.config(select_upload_type="default")

In [ ]:
my_measurement = qe.quantum_measurement(verbose=False)
my_measurement.config(select_measurement = "default")

In [ ]:
my_quantum_computer = qe.quantum_computer(verbose=False)
my_quantum_computer.config(select_qc_model="AB_repeat", depth=2) 
my_quantum_computer.inputs(my_quantum_encoding, my_entangling, my_parameterised, my_measurement)

In [ ]:
my_trainer = qe.quantum_trainer(verbose=False)
my_trainer.config(select_optimiser = "GD/SGD")
my_trainer.inputs(my_teacher_model, my_quantum_computer, my_loss_function)
my_training_report = my_trainer.train()

In [ ]:
x = np.arange(0.1, 2, 0.1)
y = my_quantum_model.predict(x)

fig, ax = plt.subplots(dpi=100)
ax.plot(x,y)